In [1]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [2]:
# import the package
import osmosispy
from osmosispy import utils, pytypes, Coin

In [3]:
# configure
mnemonic_key = "notice oak worry limit wrap speak medal online prefer cluster roof addict wrist behave treat actual wasp year salad speed social layer crew genius"

## Prepare the account

to start localnet clone the [osmosis-labs/osmosis](https://github.com/osmosis-labs/osmosis) repo and run the following command

```bash
make localnet-build
make localnet-start
```

In [4]:
# # authorize in the localnet
network = osmosispy.Network.localnet()

# authorize in the mainnet
# network = osmosispy.Network.mainnet()

trader = osmosispy.Sdk.authorize(key=mnemonic_key).with_network(network).with_config(pytypes.TxConfig())

# check the address
print(trader.address)

osmo1phaxpevm5wecex2jyaqty2a4v02qj7qmlmzk5a


## Check balances of the account

In [5]:
from osmosis_proto.proto.cosmos.bank.v1beta1 import (
  query_pb2 as bank_query,
  query_pb2_grpc as bank_query_grpc,
)

In [6]:
stub_bank = bank_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_bank.AllBalances(
        bank_query.QueryAllBalancesRequest(address=trader.address)
    )
)

print(result["balances"])


[]


## Query Pools

In [7]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
  query_pb2 as gamm_query,
  query_pb2_grpc as gamm_query_grpc,
)

In [8]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

print(result["pools"])

[]


## Query Locks

In [9]:
from osmosis_proto.proto.osmosis.lockup import (
    query_pb2 as lockup_query,
    query_pb2_grpc as lockup_query_grpc,
)

In [10]:
# Get account locks

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.AccountLockedDuration(
        # pass owner address string to query
        lockup_query.AccountLockedDurationRequest(owner=trader.address)
    )
)

print(result["locks"])

[]


### **ATTENTION**
To make it work you need to specify actual `lock_id`

In [11]:
lock_id=123

In [12]:
# Lock by ID

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.LockedByID(
        # pass lock id to query
        lockup_query.LockedRequest(lock_id=lock_id)
    )
)

print(result)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "lock with ID 123 does not exist: lockup not found"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:9090 {created_time:"2023-04-05T19:26:49.594307063+01:00", grpc_status:2, grpc_message:"lock with ID 123 does not exist: lockup not found"}"
>

## Query Epochs

In [ ]:
from osmosis_proto.proto.osmosis.epochs import (
    query_pb2 as epochs_query,
    query_pb2_grpc as epochs_query_grpc,
)

In [ ]:
stub_epochs = epochs_query_grpc.QueryStub(trader.query.chain_channel)

In [ ]:
# Get all epochs
result = utils.deserialize(
    stub_epochs.EpochInfos(
        epochs_query.QueryEpochsInfoRequest()
    )
)

print(result)

In [ ]:
# Get current epoch
result = utils.deserialize(
    stub_epochs.CurrentEpoch(
        epochs_query.QueryCurrentEpochRequest(identifier="day")
    )
)

print(result)

## Get pools

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
    query_pb2 as gamm_query,
    query_pb2_grpc as gamm_query_grpc,
)

In [ ]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pools_resp = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

# pools need to be deserialized manually as they are Any types
pools = [utils.deserialize_pool(pool) for pool in pools_resp["pools"]]

print("pools", pools)

In [ ]:
# Get specific pool by ID
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pool_raw_data = utils.deserialize(
    stub_gamm.Pool(
        # checking this pool https://app.osmosis.zone/pool/678
        gamm_query.QueryPoolRequest(pool_id=678)
    )
)

pool = utils.deserialize_pool(pool_raw_data["pool"])

print(pool)

In [ ]:
# Get spot price
price_data = utils.deserialize(
    stub_gamm.SpotPrice(
        gamm_query.QuerySpotPriceRequest(
            pool_id=pool["id"],
            base_asset_denom=pool["pool_assets"][0]["token"]["denom"],
            quote_asset_denom=pool["pool_assets"][1]["token"]["denom"],
        )
    )
)

print(price_data["spot_price"])